In [3]:
# Install dependencies jika belum ada
import subprocess
import sys

def install_package(package):
    """Install package menggunakan pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} berhasil diinstall!")
    except subprocess.CalledProcessError:
        print(f"❌ Gagal install {package}")

# Cek dan install dependencies
try:
    import qrcode
    from PIL import Image
    print("✅ Dependencies sudah tersedia!")
except ImportError:
    print("🔄 Installing dependencies...")
    install_package("qrcode[pil]")
    install_package("pillow")

    # Import ulang setelah install
    try:
        import qrcode
        from PIL import Image
        print("✅ Dependencies berhasil diinstall dan diimport!")
    except ImportError as e:
        print(f"❌ Error: {e}")
        print("💡 Coba jalankan di terminal: pip install qrcode[pil] pillow")

import os

def generate_qr_code(url, filename="qr_code.png", size=10, border=4):
    """
    Generate QR code dari URL/link yang diberikan

    Parameters:
    url (str): Link/URL yang akan di-convert ke QR code
    filename (str): Nama file output (default: qr_code.png)
    size (int): Ukuran box QR code (default: 10)
    border (int): Ukuran border QR code (default: 4)

    Returns:
    bool: True jika berhasil, False jika gagal
    """
    try:
        # Membuat instance QR code
        qr = qrcode.QRCode(
            version=1,  # Mengontrol ukuran QR code (1-40)
            error_correction=qrcode.constants.ERROR_CORRECT_L,  # Level koreksi error
            box_size=size,  # Ukuran setiap box dalam pixel
            border=border,  # Ukuran border
        )

        # Menambahkan data (URL) ke QR code
        qr.add_data(url)
        qr.make(fit=True)

        # Membuat gambar QR code
        img = qr.make_image(fill_color="black", back_color="white")

        # Menyimpan gambar
        img.save(filename)

        print(f"✅ QR Code berhasil dibuat!")
        print(f"📁 File disimpan sebagai: {filename}")
        print(f"🔗 Link: {url}")

        return True

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

def generate_qr_with_logo(url, logo_path, filename="qr_code_with_logo.png", size=10, border=4):
    """
    Generate QR code dengan logo di tengah

    Parameters:
    url (str): Link/URL yang akan di-convert ke QR code
    logo_path (str): Path ke file logo
    filename (str): Nama file output
    size (int): Ukuran box QR code
    border (int): Ukuran border QR code

    Returns:
    bool: True jika berhasil, False jika gagal
    """
    try:
        # Membuat QR code dasar
        qr = qrcode.QRCode(
            version=1,
            error_correction=qrcode.constants.ERROR_CORRECT_H,  # High error correction untuk logo
            box_size=size,
            border=border,
        )

        qr.add_data(url)
        qr.make(fit=True)

        # Membuat gambar QR code
        qr_img = qr.make_image(fill_color="black", back_color="white")

        # Membuka dan resize logo
        logo = Image.open(logo_path)

        # Menghitung ukuran logo (10% dari ukuran QR code)
        qr_width, qr_height = qr_img.size
        logo_size = min(qr_width, qr_height) // 5

        # Resize logo
        logo = logo.resize((logo_size, logo_size), Image.Resampling.LANCZOS)

        # Menghitung posisi tengah untuk logo
        logo_pos = ((qr_width - logo_size) // 2, (qr_height - logo_size) // 2)

        # Menempel logo ke QR code
        qr_img.paste(logo, logo_pos)

        # Menyimpan gambar
        qr_img.save(filename)

        print(f"✅ QR Code dengan logo berhasil dibuat!")
        print(f"📁 File disimpan sebagai: {filename}")
        print(f"🔗 Link: {url}")

        return True

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

def batch_generate_qr(urls_list, prefix="qr_"):
    """
    Generate multiple QR codes dari list URL

    Parameters:
    urls_list (list): List berisi URL-URL
    prefix (str): Prefix untuk nama file

    Returns:
    int: Jumlah QR code yang berhasil dibuat
    """
    success_count = 0

    for i, url in enumerate(urls_list, 1):
        filename = f"{prefix}{i}.png"
        if generate_qr_code(url, filename):
            success_count += 1
        print("-" * 50)

    print(f"📊 Total QR code berhasil dibuat: {success_count}/{len(urls_list)}")
    return success_count

# Contoh penggunaan
if __name__ == "__main__":
    print("🔄 QR Code Generator untuk Google Forms")
    print("=" * 50)

    # URL Google Forms yang sudah ditanamkan
    google_form_url = "https://www.youtube.com/watch?v=6EEW-9NDM5k&list=RD-_Av7re3Ce8&index=4"

    print(f"📝 Link Google Forms: {google_form_url}")
    print()

    # Pilihan mode
    print("Pilih mode:")
    print("1. Generate QR untuk Google Forms (sudah tertanam)")
    print("2. Generate QR untuk link custom")

    mode = input("Pilihan (1-2, Enter untuk mode 1): ").strip()

    if mode == "2":
        # Mode custom URL
        url = input("Masukkan URL/Link custom: ")
        if not url.strip():
            print("❌ URL tidak boleh kosong!")
            exit()
    else:
        # Mode Google Forms (default)
        url = google_form_url
        print("✅ Menggunakan Google Forms yang sudah tertanam")

    # Pilihan ukuran
    print("\nPilih ukuran QR Code:")
    print("1. Kecil (box_size=5)")
    print("2. Sedang (box_size=10) - Default")
    print("3. Besar (box_size=15)")

    size_choice = input("Pilihan (1-3, Enter untuk default): ").strip()

    if size_choice == "1":
        box_size = 5
    elif size_choice == "3":
        box_size = 15
    else:
        box_size = 10

    # Generate QR code
    if mode == "1" or not mode:
        default_filename = "google_forms_qr.png"
    else:
        default_filename = "qr_code.png"

    filename = input(f"Nama file output (Enter untuk default '{default_filename}'): ").strip()
    if not filename:
        filename = default_filename

    # Pastikan ekstensi .png
    if not filename.endswith('.png'):
        filename += '.png'

    success = generate_qr_code(url, filename, box_size)

    if success:
        print(f"\n🎉 QR Code siap digunakan!")
        if url == google_form_url:
            print(f"📋 QR Code untuk Google Forms telah dibuat!")
            print(f"💡 Scan QR code untuk mengisi formulir")
        else:
            print(f"💡 Scan QR code untuk membuka: {url}")

    # Contoh penggunaan langsung untuk Google Forms
    """
    # Uncomment untuk langsung generate QR Google Forms
    google_form_url = "https://docs.google.com/forms/d/e/1FAIpQLSeiEF9DgeUaQmQM1M7gStL8Xx4mg28GP1eGH8-Yxj3MhCAleg/viewform?usp=header"
    generate_qr_code(google_form_url, "google_forms_qr.png", size=12, border=4)
    """

    # Contoh dengan logo
    """
    # Uncomment jika ingin menambahkan logo
    logo_path = "logo.png"  # Ganti dengan path logo Anda
    if os.path.exists(logo_path):
        generate_qr_with_logo(url, logo_path)
    """

✅ Dependencies sudah tersedia!
🔄 QR Code Generator untuk Google Forms
📝 Link Google Forms: https://www.youtube.com/watch?v=6EEW-9NDM5k&list=RD-_Av7re3Ce8&index=4

Pilih mode:
1. Generate QR untuk Google Forms (sudah tertanam)
2. Generate QR untuk link custom
Pilihan (1-2, Enter untuk mode 1): 2
Masukkan URL/Link custom: https://www.youtube.com/watch?v=6EEW-9NDM5k&list=RD-_Av7re3Ce8&index=4

Pilih ukuran QR Code:
1. Kecil (box_size=5)
2. Sedang (box_size=10) - Default
3. Besar (box_size=15)
Pilihan (1-3, Enter untuk default): 2
Nama file output (Enter untuk default 'qr_code.png'): gg
✅ QR Code berhasil dibuat!
📁 File disimpan sebagai: gg.png
🔗 Link: https://www.youtube.com/watch?v=6EEW-9NDM5k&list=RD-_Av7re3Ce8&index=4

🎉 QR Code siap digunakan!
📋 QR Code untuk Google Forms telah dibuat!
💡 Scan QR code untuk mengisi formulir


In [11]:
import subprocess
import sys

def install_package(package):
    """Install package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully!")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package}")

# Check and install dependencies
try:
    import qrcode
    from PIL import Image
    print("Dependencies are already available!")
except ImportError:
    print("Installing dependencies...")
    install_package("qrcode[pil]")
    install_package("pillow")

    # Re-import after installation
    try:
        import qrcode
        from PIL import Image
        print("Dependencies installed and imported successfully!")
    except ImportError as e:
        print(f"Error: {e}")
        print("Try running in terminal: pip install qrcode[pil] pillow")

import os
import argparse

def generate_qr_code(url, filename="qr_code.png", size=10, border=4, error_correction='L', fill_color="black", back_color="white", verbose=True):
    """
    Generate QR code from the given URL/link

    Parameters:
    url (str): The URL to convert to QR code
    filename (str): Output file name (default: qr_code.png)
    size (int): Box size of QR code (default: 10)
    border (int): Border size of QR code (default: 4)
    error_correction (str): Error correction level ('L', 'M', 'Q', 'H') (default: 'L')
    fill_color (str): Fill color (default: black)
    back_color (str): Background color (default: white)
    verbose (bool): Print status messages (default: True)

    Returns:
    bool: True if successful, False if failed
    """
    try:
        error_levels = {
            'L': qrcode.constants.ERROR_CORRECT_L,
            'M': qrcode.constants.ERROR_CORRECT_M,
            'Q': qrcode.constants.ERROR_CORRECT_Q,
            'H': qrcode.constants.ERROR_CORRECT_H
        }
        if error_correction not in error_levels:
            raise ValueError("Invalid error correction level. Choose from 'L', 'M', 'Q', 'H'.")

        # Create QR code instance
        qr = qrcode.QRCode(
            version=None,  # Auto-detect version
            error_correction=error_levels[error_correction],
            box_size=size,
            border=border,
        )

        # Add data (URL) to QR code
        qr.add_data(url)
        qr.make(fit=True)

        # Create QR code image
        img = qr.make_image(fill_color=fill_color, back_color=back_color)

        # Save image
        img.save(filename)

        if verbose:
            print(f"QR Code generated successfully!")
            print(f"File saved as: {filename}")
            print(f"URL: {url}")

        return True

    except Exception as e:
        if verbose:
            print(f"Error: {str(e)}")
        return False

def generate_qr_with_logo(url, logo_path, filename="qr_code_with_logo.png", size=10, border=4, error_correction='H', fill_color="black", back_color="white", logo_scale=0.2, verbose=True):
    """
    Generate QR code with a logo in the center

    Parameters:
    url (str): The URL to convert to QR code
    logo_path (str): Path to the logo file
    filename (str): Output file name
    size (int): Box size of QR code
    border (int): Border size of QR code
    error_correction (str): Error correction level ('L', 'M', 'Q', 'H') (default: 'H')
    fill_color (str): Fill color (default: black)
    back_color (str): Background color (default: white)
    logo_scale (float): Scale of logo relative to QR size (default: 0.2)
    verbose (bool): Print status messages (default: True)

    Returns:
    bool: True if successful, False if failed
    """
    try:
        if not os.path.exists(logo_path):
            raise FileNotFoundError(f"Logo file not found: {logo_path}")

        error_levels = {
            'L': qrcode.constants.ERROR_CORRECT_L,
            'M': qrcode.constants.ERROR_CORRECT_M,
            'Q': qrcode.constants.ERROR_CORRECT_Q,
            'H': qrcode.constants.ERROR_CORRECT_H
        }
        if error_correction not in error_levels:
            raise ValueError("Invalid error correction level. Choose from 'L', 'M', 'Q', 'H'.")

        # Create base QR code
        qr = qrcode.QRCode(
            version=None,
            error_correction=error_levels[error_correction],
            box_size=size,
            border=border,
        )

        qr.add_data(url)
        qr.make(fit=True)

        # Create QR code image
        qr_img = qr.make_image(fill_color=fill_color, back_color=back_color).convert("RGBA")

        # Open and resize logo
        logo = Image.open(logo_path).convert("RGBA")

        # Calculate logo size
        qr_width, qr_height = qr_img.size
        logo_size = int(min(qr_width, qr_height) * logo_scale)

        # Resize logo
        logo = logo.resize((logo_size, logo_size), Image.Resampling.LANCZOS)

        # Calculate center position for logo
        logo_pos = ((qr_width - logo_size) // 2, (qr_height - logo_size) // 2)

        # Paste logo onto QR code
        qr_img.paste(logo, logo_pos, logo)

        # Save image
        qr_img.save(filename)

        if verbose:
            print(f"QR Code with logo generated successfully!")
            print(f"File saved as: {filename}")
            print(f"URL: {url}")

        return True

    except Exception as e:
        if verbose:
            print(f"Error: {str(e)}")
        return False

def batch_generate_qr(urls_list, prefix="qr_", size=10, border=4, error_correction='L', fill_color="black", back_color="white", verbose=True):
    """
    Generate multiple QR codes from a list of URLs

    Parameters:
    urls_list (list): List of URLs
    prefix (str): Prefix for file names
    size (int): Box size of QR code
    border (int): Border size of QR code
    error_correction (str): Error correction level
    fill_color (str): Fill color
    back_color (str): Background color
    verbose (bool): Print status messages

    Returns:
    int: Number of QR codes successfully created
    """
    success_count = 0

    for i, url in enumerate(urls_list, 1):
        filename = f"{prefix}{i}.png"
        if generate_qr_code(url, filename, size, border, error_correction, fill_color, back_color, verbose):
            success_count += 1
        if verbose:
            print("-" * 50)

    if verbose:
        print(f"Total QR codes generated: {success_count}/{len(urls_list)}")
    return success_count

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Advanced QR Code Generator")
    parser.add_argument("--url", type=str, help="URL to generate QR for")
    parser.add_argument("--urls-file", type=str, help="File containing list of URLs (one per line) for batch generation")
    parser.add_argument("--logo", type=str, help="Path to logo file for embedding")
    parser.add_argument("--output", type=str, default="qr_code.png", help="Output file name")
    parser.add_argument("--size", type=int, default=10, help="Box size (default: 10)")
    parser.add_argument("--border", type=int, default=4, help="Border size (default: 4)")
    parser.add_argument("--error-correction", type=str, default='L', choices=['L', 'M', 'Q', 'H'], help="Error correction level (default: L)")
    parser.add_argument("--fill-color", type=str, default="black", help="Fill color (default: black)")
    parser.add_argument("--back-color", type=str, default="white", help="Background color (default: white)")
    parser.add_argument("--logo-scale", type=float, default=0.2, help="Logo scale relative to QR size (default: 0.2)")
    parser.add_argument("--prefix", type=str, default="qr_", help="Prefix for batch file names (default: qr_)")
    parser.add_argument("--quiet", action="store_true", help="Suppress status messages")

    args, unknown = parser.parse_known_args()
    verbose = not args.quiet

    if args.url is None and args.urls_file is None:
        print("No URL or URLs file provided. Entering interactive mode.")

        mode = input("Choose mode (1: single URL, 2: batch from file): ").strip()

        if mode == "1":
            args.url = input("Enter URL: ").strip()
            if not args.url:
                print("URL is required.")
                sys.exit(1)

            args.logo = input("Enter logo path (optional): ").strip() or None
            args.output = input(f"Enter output filename (default: {args.output}): ").strip() or args.output
            size_str = input(f"Enter box size (default: {args.size}): ").strip()
            args.size = int(size_str) if size_str else args.size
            border_str = input(f"Enter border size (default: {args.border}): ").strip()
            args.border = int(border_str) if border_str else args.border
            args.error_correction = input(f"Enter error correction level (L/M/Q/H, default: {args.error_correction}): ").strip() or args.error_correction
            args.fill_color = input(f"Enter fill color (default: {args.fill_color}): ").strip() or args.fill_color
            args.back_color = input(f"Enter background color (default: {args.back_color}): ").strip() or args.back_color
            if args.logo:
                logo_scale_str = input(f"Enter logo scale (default: {args.logo_scale}): ").strip()
                args.logo_scale = float(logo_scale_str) if logo_scale_str else args.logo_scale

        elif mode == "2":
            args.urls_file = input("Enter path to URLs file: ").strip()
            if not args.urls_file:
                print("URLs file path is required.")
                sys.exit(1)

            args.prefix = input(f"Enter prefix for batch files (default: {args.prefix}): ").strip() or args.prefix
            size_str = input(f"Enter box size (default: {args.size}): ").strip()
            args.size = int(size_str) if size_str else args.size
            border_str = input(f"Enter border size (default: {args.border}): ").strip()
            args.border = int(border_str) if border_str else args.border
            args.error_correction = input(f"Enter error correction level (L/M/Q/H, default: {args.error_correction}): ").strip() or args.error_correction
            args.fill_color = input(f"Enter fill color (default: {args.fill_color}): ").strip() or args.fill_color
            args.back_color = input(f"Enter background color (default: {args.back_color}): ").strip() or args.back_color

        else:
            print("Invalid mode selected.")
            sys.exit(1)

    if args.urls_file:
        try:
            with open(args.urls_file, 'r') as f:
                urls = [line.strip() for line in f if line.strip()]
            if not urls:
                print("No URLs found in file.")
                sys.exit(1)
            batch_generate_qr(urls, args.prefix, args.size, args.border, args.error_correction, args.fill_color, args.back_color, verbose)
        except Exception as e:
            print(f"Error reading file: {e}")
    elif args.url:
        if args.logo:
            generate_qr_with_logo(args.url, args.logo, args.output, args.size, args.border, args.error_correction, args.fill_color, args.back_color, args.logo_scale, verbose)
        else:
            generate_qr_code(args.url, args.output, args.size, args.border, args.error_correction, args.fill_color, args.back_color, verbose)
    else:
        print("No valid input provided.")
        parser.print_help()
        sys.exit(1)

Dependencies are already available!
No URL or URLs file provided. Entering interactive mode.
Choose mode (1: single URL, 2: batch from file): 1
Enter URL: https://www.youtube.com/watch?v=fPO76Jlnz6c&list=RD-_Av7re3Ce8&index=26
Enter logo path (optional): 
Enter output filename (default: qr_code.png): 
Enter box size (default: 10): 10
Enter border size (default: 4): 4
Enter error correction level (L/M/Q/H, default: L): M
Enter fill color (default: black): red
Enter background color (default: white): blue
QR Code generated successfully!
File saved as: qr_code.png
URL: https://www.youtube.com/watch?v=fPO76Jlnz6c&list=RD-_Av7re3Ce8&index=26


In [9]:
#!/usr/bin/env python3
"""
Advanced QR Code Generator (fixed for Jupyter/Colab)

Default URL is set to:
https://www.youtube.com/watch?v=pvrkfb1C4tE&list=RD-_Av7re3Ce8&index=14

Usage examples (terminal):
  python qr_generator.py --url "https://example.com" --output my_qr.png
  python qr_generator.py --urls-file urls.txt --prefix batch_
"""

from __future__ import annotations
import subprocess
import sys
import os
import argparse

# -------------------------
# Optional dependency installer (best-effort)
# -------------------------
def install_package(package: str) -> None:
    """Install a package using pip (best-effort)."""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully.")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package}. Please install manually.")

# Try import qrcode and PIL, attempt to install if missing
try:
    import qrcode
    from PIL import Image
except Exception:
    print("Required packages not found. Attempting to install qrcode[pil] and pillow...")
    install_package("qrcode[pil]")
    install_package("pillow")
    try:
        import qrcode
        from PIL import Image
    except Exception as e:
        print(f"Import error after install attempt: {e}")
        print("Please install manually: pip install qrcode[pil] pillow")
        sys.exit(1)

# -------------------------
# QR generation helpers
# -------------------------
def _get_error_level(code: str):
    levels = {
        'L': qrcode.constants.ERROR_CORRECT_L,
        'M': qrcode.constants.ERROR_CORRECT_M,
        'Q': qrcode.constants.ERROR_CORRECT_Q,
        'H': qrcode.constants.ERROR_CORRECT_H,
    }
    return levels.get(code.upper())

def generate_qr_code(
    url: str,
    filename: str = "qr_code.png",
    size: int = 10,
    border: int = 4,
    error_correction: str = 'L',
    fill_color: str = "black",
    back_color: str = "white",
    verbose: bool = True
) -> bool:
    """
    Generate a QR code from the provided URL and save to filename.
    """
    try:
        err_level = _get_error_level(error_correction)
        if err_level is None:
            raise ValueError("Invalid error correction level. Choose from 'L','M','Q','H'.")

        qr = qrcode.QRCode(
            version=None,
            error_correction=err_level,
            box_size=int(size),
            border=int(border)
        )
        qr.add_data(url)
        qr.make(fit=True)

        img = qr.make_image(fill_color=fill_color, back_color=back_color)

        # Ensure PIL Image and save
        try:
            pil_img = img.convert("RGB")
        except Exception:
            pil_img = img  # already PIL image

        pil_img.save(filename)
        if verbose:
            print("QR code generated successfully.")
            print(f"Saved to: {filename}")
            print(f"URL encoded: {url}")
        return True

    except Exception as e:
        if verbose:
            print(f"Error generating QR: {e}")
        return False

def generate_qr_with_logo(
    url: str,
    logo_path: str,
    filename: str = "qr_code_with_logo.png",
    size: int = 10,
    border: int = 4,
    error_correction: str = 'H',
    fill_color: str = "black",
    back_color: str = "white",
    logo_scale: float = 0.2,
    verbose: bool = True
) -> bool:
    """
    Generate QR code and embed a centered logo.
    logo_scale is fraction of the QR's minimum side (e.g., 0.2 -> logo = 20%).
    """
    try:
        if not os.path.exists(logo_path):
            raise FileNotFoundError(f"Logo file not found: {logo_path}")

        err_level = _get_error_level(error_correction)
        if err_level is None:
            raise ValueError("Invalid error correction level. Choose from 'L','M','Q','H'.")

        qr = qrcode.QRCode(
            version=None,
            error_correction=err_level,
            box_size=int(size),
            border=int(border)
        )
        qr.add_data(url)
        qr.make(fit=True)

        qr_img = qr.make_image(fill_color=fill_color, back_color=back_color).convert("RGBA")

        logo = Image.open(logo_path).convert("RGBA")

        qr_w, qr_h = qr_img.size
        logo_size = int(min(qr_w, qr_h) * float(logo_scale))
        if logo_size <= 0:
            raise ValueError("Computed logo size <= 0. Check logo_scale value.")

        # Resampling compatibility across Pillow versions
        try:
            resample = Image.Resampling.LANCZOS  # Pillow >= 9.1.0
        except AttributeError:
            resample = Image.LANCZOS  # older Pillow

        logo = logo.resize((logo_size, logo_size), resample)

        pos = ((qr_w - logo_size) // 2, (qr_h - logo_size) // 2)
        qr_img.paste(logo, pos, logo)  # use logo as alpha mask

        final_img = qr_img.convert("RGB")
        final_img.save(filename)

        if verbose:
            print("QR code with logo generated successfully.")
            print(f"Saved to: {filename}")
            print(f"URL encoded: {url}")
        return True

    except Exception as e:
        if verbose:
            print(f"Error generating QR with logo: {e}")
        return False

def batch_generate_qr(
    urls_list,
    prefix: str = "qr_",
    size: int = 10,
    border: int = 4,
    error_correction: str = 'L',
    fill_color: str = "black",
    back_color: str = "white",
    verbose: bool = True
) -> int:
    success_count = 0
    for i, url in enumerate(urls_list, start=1):
        filename = f"{prefix}{i}.png"
        ok = generate_qr_code(url, filename, size, border, error_correction, fill_color, back_color, verbose)
        if ok:
            success_count += 1
        if verbose:
            print("-" * 40)
    if verbose:
        print(f"Total: {success_count}/{len(urls_list)} generated.")
    return success_count

# -------------------------
# Default URL (provided)
# -------------------------
DEFAULT_URL = "https://www.youtube.com/watch?v=pvrkfb1C4tE&list=RD-_Av7re3Ce8&index=14"

# -------------------------
# CLI entrypoint
# -------------------------
def main():
    parser = argparse.ArgumentParser(description="Advanced QR Code Generator")
    parser.add_argument("--url", type=str, default=DEFAULT_URL, help="URL to generate QR for (default: provided YouTube link)")
    parser.add_argument("--urls-file", type=str, help="Path to a file with URLs (one per line) for batch generation")
    parser.add_argument("--logo", type=str, help="Path to logo image to embed")
    parser.add_argument("--output", type=str, default="qr_code.png", help="Output filename")
    parser.add_argument("--size", type=int, default=10, help="Box size (default: 10)")
    parser.add_argument("--border", type=int, default=4, help="Border size (default: 4)")
    parser.add_argument("--error-correction", type=str, default='L', choices=['L','M','Q','H'], help="Error correction level")
    parser.add_argument("--fill-color", type=str, default="black", help="QR fill color")
    parser.add_argument("--back-color", type=str, default="white", help="QR background color")
    parser.add_argument("--logo-scale", type=float, default=0.2, help="Logo scale relative to QR min dimension (default 0.2)")
    parser.add_argument("--prefix", type=str, default="qr_", help="Prefix for batch files")
    parser.add_argument("--quiet", action="store_true", help="Suppress console messages")

    # Use parse_known_args so Jupyter/Colab injected args (e.g. -f /path/kernel-*.json) won't cause SystemExit
    args, unknown = parser.parse_known_args()
    verbose = not args.quiet

    if verbose and unknown:
        print("Note: ignored unknown args:", unknown)

    # Batch mode
    if args.urls_file:
        try:
            with open(args.urls_file, "r", encoding="utf-8") as f:
                urls = [line.strip() for line in f if line.strip()]
        except Exception as e:
            print(f"Error reading URLs file: {e}")
            sys.exit(1)
        if not urls:
            print("No URLs found in the provided file.")
            sys.exit(1)
        batch_generate_qr(urls, args.prefix, args.size, args.border, args.error_correction, args.fill_color, args.back_color, verbose)
        return

    # Single URL (uses default if none provided)
    if args.url:
        if args.logo:
            ok = generate_qr_with_logo(
                args.url,
                args.logo,
                args.output,
                args.size,
                args.border,
                args.error_correction,
                args.fill_color,
                args.back_color,
                args.logo_scale,
                verbose
            )
        else:
            ok = generate_qr_code(
                args.url,
                args.output,
                args.size,
                args.border,
                args.error_correction,
                args.fill_color,
                args.back_color,
                verbose
            )
        if not ok:
            sys.exit(1)
    else:
        # Should not happen because --url has a default, but kept for clarity
        print("No URL provided. Use --url or --urls-file.")
        parser.print_help()
        sys.exit(1)

if __name__ == "__main__":
    main()


Note: ignored unknown args: ['-f', '/root/.local/share/jupyter/runtime/kernel-8ff0fc53-04a6-404b-b39b-35f0af8b8c1a.json']
QR code generated successfully.
Saved to: qr_code.png
URL encoded: https://www.youtube.com/watch?v=pvrkfb1C4tE&list=RD-_Av7re3Ce8&index=14


In [ ]:
from google.colab import drive
drive.mount('/content/drive')